# ENO 和 WENO

ENO 和 WENO 在处理间断解的问题时，两种方法都可以客服非物理震荡. ENO 是本质非振荡，WENO 是加权本质不振荡，但是 WENO 既能在光滑区域保持高阶精度又能处理间断.

选取左差和右差是根据速度场选择，用左差，右差都是一阶，为了得到高阶用多个模板点，引入ENO，给出模板光滑度来衡量.

光滑因子的表达式:

$$
\alpha = \frac{d_r}{\beta_r+\epsilon}
$$

$$
\beta_r = \sum_{k-1}^{i=1}\int_{x_{i-\frac{1}{2}}}^{x_{i+\frac{1}{2}}}\Delta x^{2l-1}(\frac{\partial^lp_r(x)}{\partial^lx})\mathrm{d}x
$$

光滑因子越小，越光滑，因此选用光滑因子最小的.$p_r(x)$ 是模板点构造的插值多项式.

把所有模板点的信息都用上做一个凸组合，，得到一个高阶的精度的插值格式.例:五个点可以构造 $5$ 阶精度， 

$$
\sum_{i=-2}^{2}a_iu_i=d_1\sum_{i=-2}^{0}b_i^1u_i+d_2\sum_{i=-1}^{1}b_i^2u_i+d_3\sum_{i=0}^{2}b_i^3u_i
$$ ,

三个模板点 $\sum_{i=-2}^{0}b_i^1u_i,\sum_{i=-1}^{1}b_i^2u_i,\sum_{i=0}^{2}b_i^3u_i$可以确定一个组合系数，用光滑因子调整权系数 $d_1,d_2,d_3$

ENO 的思想是尽可能的避免模板点包含间断的单元；假设需要找到 $k+1$ 个连续点的模板，这个模板包含点 $x_{i-\frac{1}{2}}$ 和 $x_{i+\frac{1}{2}}$，使得函数在这个模板比其他可能的模板上是最光滑的.通过增加两边的端点来得到一个二次插值，来提高阶数. 即选择一个最好的模板点，但是由于这个点也只是可由三个点表示，所以阶数最高为3! WENO 是所有模板点的凸组合代替仅用一个模板点，这样就提高了它的阶数，WENO 的关键是对于权重 $\omega_r$ 的选择. 要求 $\omega_r\geq0,~\sum_{r=0}^{k-1}\omega_r=1$

**ENO的重构**

1. 计算初始函数 $V(x)$ 的差商
1. 用对于 $V(x)$ 的两个模板点 $S_2(i)=\{x_{i-\frac{1}{2}},x_{i+\frac{1}{2}}\}$,它等于对于 $\overline{v}$ 上的一个模板点 $S_1(i) = \{I_i\}$
这里 $V(x)$ 是 $v(x)$ 原函数， $\overline{v}$ 是 $v(x)$ 的积分平均， $\overline{v}=\frac{1}{\Delta x_i}\int_{x_{i-\frac{1}{2}}}^{x_{i+\frac{1}{2}}}v(\xi)\mathrm{d}\xi$
1. 对于 $l = 2,\cdots,k$ ,假设 
$$
S_l(i)=\{x_{j+\frac{1}{2}},\cdots,x_{j+l-\frac{1}{2}}\}
$$
已知，增加两个点 $x_{j-\frac{1}{2}},x_{j+\frac{1}{2}}$, 如果
$$
|V[x_{j-\frac{1}{2}},\cdots,x_{j+l-\frac{1}{2}}]|<|V[x_{j+\frac{1}{2}},\cdots,x_{j+l+\frac{1}{2}}]|
$$
增加 $x_{j-\frac{1}{2}}$ 到模板 $S_l(i)$ 得到 $S_{l+1}(i)=\{x_{j-\frac{1}{2}},\cdots,x_{j+l-\frac{1}{2}}\}$
否则，增加 $x_{j+l+\frac{1}{2}}$ 得到 $S_{l+1}(i)=\{x_{j+\frac{1}{2}},\cdots,x_{j+l+\frac{1}{2}}\}$
1.  用拉格朗日或者牛顿格式得到 $p_i(x)$ ，这样可以用 $p_i(x)$ 得到在单元边界的近似
$$
v_{i+\frac{1}{2}}^{-} = p_i(x_{i+\frac{1}{2}})\\
v_{i-\frac{1}{2}}^{+} = p_i(x_{i-\frac{1}{2}})
$$

**WENO** 

假设 $k$ 个模板点 

$$
S_r(i) =\{x_{i-r},\cdots,x_{i-r+k-1}\},r=0,\cdots,k-1
$$

对于 $v_{i+1/2}$ 的值产生 $k$ 个不同的 重构.

根据 

$$
v_{i+1/2}^{(r)} = \sum_{j=0}^{k-1}c_{rj}\overline{v}_{i-r+j},r=0,\cdots,k-1
$$

WENO重构是所有$v_{i+1/2}^{(r)}$　凸组合.边界上$v_{i+1/2}$的值

$$
v_{i+1/2} = \sum_{j=0}^{k-1}w_{r}\overline{v}_{i+1/2}^{(r)},r=0,\cdots,k-1
$$

关键是选取 $w_r$ ，要求 $\omega_r\geq0,~\sum_{r=0}^{k-1}\omega_r=1$

**ＷENO的重构**

1.从　$v_{i+1/2}^{(r)} = \sum_{j=0}^{k-1}c_{rj}\overline{v}_{i-r+j},r=0,\cdots,k-1$ ,得到 ${v}_{i+1/2}^{(r)}$ 的值和 $k$ 阶精度

2.找一个常数 $d_r,\overline d_r$ 使得

$$
v_{i-1/2} = \sum_{j=0}^{k-1}\overline d_{r}{v}_{i-1/2}^{(r)}+O(\Delta x^{2k-1})
$$

$$
\overline d_r=d_{k-1-r}
$$

3.找一个光滑指标 $\beta_r$

4.用 

$$
w_r = \frac{\alpha_r}{\sum_{s=0}^{k-1}\alpha_s},r=0,\cdots,k-1\\
\alpha_r = \frac{d_r}{(\epsilon+\beta_r)^2}
$$

和

$$
\overline w_r = \frac{\overline\alpha_r}{\sum_{s=0}^{k-1}\overline\alpha_s},r=0,\cdots,k-1\\
\overline\alpha_r = \frac{\overline d_r}{(\epsilon+\beta_r)^2}
$$

5.找处 $2k-1$ 阶的重构

$$
v_{i+1/2}^- = \sum_{r=0}^{k-1}w_{r}{v}_{i+1/2}^{(r)}\\
v_{i-1/2}^+ = \sum_{r=0}^{k-1}\overline w_{r}{v}_{i-1/2}^{(r)}
$$
